In [37]:
!pip install -i http://pypi.douban.com/simple --trusted-host pypi.douban.com opencv-contrib-python==3.4.2.17


Looking in indexes: http://pypi.douban.com/simple


In [74]:
#Feature Extraction: de=0 then SURF
#Feature Extraction: de any other value then SIFT
#KMeans:cannot use huge number of descriptors


import cv2
import os
from pathlib import Path
from matplotlib import pyplot as plt
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
def makedataset(de,km):
  noi = 0
  dl=[]
  kpl=[]
  data = []
  label=[]
  tdata=[]
  tlabel=[]
  rawImg =[]
  for t in range(1,11):
    k=t
    if(t==10):
      t="00010"+"/train.txt"
      fol="00010"
      m="00010"+"/test.txt"
    else:
      fol="0000"+str(t)
      t="0000"+str(t)+"/train.txt"
      m="0000"+str(k)+"/test.txt"
      
    
    fpath = os.path.join("/content/drive/MyDrive/CS893 Sp2022 A1 Dataset/",t)
    tpath = os.path.join("/content/drive/MyDrive/CS893 Sp2022 A1 Dataset/",m)
    f = open(fpath, "r")
    te= open(tpath,"r")
    if(de==0):
      sift = cv2.xfeatures2d.SURF_create(300)
    else:
      sift = cv2.xfeatures2d.SIFT_create()



    for x in f:
      path = os.path.join("/content/drive/MyDrive/CS893 Sp2022 A1 Dataset/",fol.strip(),x.strip())
      img = cv2.imread(path)
      gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      kp, des = sift.detectAndCompute(gray_img, None)
      
      dl.append(des)
      kpl.append(kp)
      if len(kp)!=0:
        for d in des:
          data.append(d)        
      noi=noi+1
      label.append(k)
    f.close()
    for x in te:
      path = os.path.join("/content/drive/MyDrive/CS893 Sp2022 A1 Dataset/",fol.strip(),x.strip())
      img = cv2.imread(path)
      tdata.append(img)
      tlabel.append(k)
  batch_size = 200
  kmeans = MiniBatchKMeans(n_clusters=km, batch_size=batch_size, verbose=1).fit(data)

  return dl,label,tdata,tlabel,sift,data,kpl,kmeans


In [75]:
#creation of histogram which will be used as feature for training dataset
import numpy as np
from sklearn.cluster import MiniBatchKMeans
def featureCraetion(dL,k):
   kmeans.verbose = False
   histo_list = []
   for des in dl:
      histo = np.zeros(k)
      nkp = np.size(kpl)
      if type(des) != type(None):
        for d in des:
          idx = kmeans.predict([d])
          histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly
      histo_list.append(histo)
   return histo_list

In [76]:
#creation of histogram which will be used as feature for testing dataset
def creaturetrainingfeature(td,k):
  X_List=[]
  for d in td:
    kp, des = sift.detectAndCompute(d, None)
    x = np.zeros(k)
    nkp = np.size(kp)
    if len(kp)!=0:
      for d in des:
        idx = kmeans.predict([d])
        x[idx] += 1/nkp
    X_List.append(x)
  return X_List


In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn import metrics
def classification(histo_list,label,X_List,tlabel):
  #Create a Gaussian Classifier
  clf=RandomForestClassifier(n_estimators=130)

  #Train the model using the training sets y_pred=clf.predict(X_test)
  clf.fit(histo_list, label)

  y_pred=clf.predict(X_List)
  cm = confusion_matrix(tlabel, y_pred)
  print("Classifiaction Report using RandomForest")
  print(metrics.classification_report(tlabel, y_pred, digits=3))
  gnb = GaussianNB().fit(histo_list, label)
  gnb_predictions = gnb.predict(X_List)
 
  cm = confusion_matrix(tlabel, gnb_predictions)
  t=metrics.classification_report(tlabel, gnb_predictions, digits=2)
  print(t)

In [ ]:
#SURF Feature extraction
[dl,label,tdata,tlabel,sift,data,kpl,kmeans]= makedataset(0,900)
histo_list=featureCraetion(dl,900)
X_List = creaturetrainingfeature(tdata,900)


In [71]:
print("SURF Feature Extraction")
classification(histo_list,label,X_List,tlabel)


SURF Feature Extraction
Classifiaction Report using RandomForest
              precision    recall  f1-score   support

           1      0.944     0.850     0.895        20
           2      0.952     1.000     0.976        20
           3      0.947     0.900     0.923        20
           4      1.000     1.000     1.000        20
           5      0.952     1.000     0.976        20
           6      0.870     1.000     0.930        20
           7      0.750     0.900     0.818        20
           8      1.000     0.700     0.824        20
           9      0.909     1.000     0.952        20
          10      0.889     0.800     0.842        20

    accuracy                          0.915       200
   macro avg      0.921     0.915     0.914       200
weighted avg      0.921     0.915     0.914       200

              precision    recall  f1-score   support

           1       0.58      0.55      0.56        20
           2       0.80      0.40      0.53        20
           3 

In [ ]:
[dl,label,tdata,tlabel,sift,data,kpl,kmeans]= makedataset(1,2000)
histo_list=featureCraetion(dl,2000)
X_List = creaturetrainingfeature(tdata,2000)



In [73]:
print("SIFT Feauture Extraction With Classification")
classification(histo_list,label,X_List,tlabel)

SIFT Feauture Extraction With Classification
Classifiaction Report using RandomForest
              precision    recall  f1-score   support

           1      0.947     0.900     0.923        20
           2      1.000     1.000     1.000        20
           3      1.000     0.950     0.974        20
           4      1.000     1.000     1.000        20
           5      0.714     0.750     0.732        20
           6      0.889     0.800     0.842        20
           7      0.667     0.600     0.632        20
           8      0.947     0.900     0.923        20
           9      0.952     1.000     0.976        20
          10      0.640     0.800     0.711        20

    accuracy                          0.870       200
   macro avg      0.876     0.870     0.871       200
weighted avg      0.876     0.870     0.871       200

              precision    recall  f1-score   support

           1       0.67      0.70      0.68        20
           2       1.00      0.90      0.95   